In [ ]:
# This chunk of code make it possible to use src functions
import sys
import os
module_path = os.path.abspath(os.path.join('../src'))

if module_path not in sys.path:
    sys.path.append(module_path)
import warnings
warnings.filterwarnings('ignore')
    
from DSProject import DSProject
import pandas as pd
from model import *

dsp = DSProject()

In [ ]:
df = dsp.read_table(train_dataset, 'feature')

Check training data prior to preprocessing

In [ ]:
print(df['Group'].value_counts()*100/len(df.index))

In [ ]:
print(df['Range_day'].value_counts())

In [ ]:
df = df.loc[df['Group'].isin([1,3,5])]
df = df.loc[df['Range_day']==1]

In [ ]:
total = len(df.index)
missing_columns = []
for column in df.columns:
    if df[column].isnull().sum()/total > 0.80:
        missing_columns.append(column)
    if len(df[column].unique().tolist()) < 2:
        missing_columns.append(column)
print(missing_columns)
df = df.drop(missing_columns, axis=1)

In [ ]:
col_to_drop = list(set(df.columns).intersection(set(index_columns + delete_columns + date_columns)))
print(col_to_drop)
df = df.drop(col_to_drop, axis=1)

In [ ]:
def get_intersect_columns(df, column_list):
    return list(set(df.columns).intersection(set(column_list)))
category_columns = get_intersect_columns(df, category_columns)
numeric_columns = get_intersect_columns(df, numeric_columns)
min_max_columns = get_intersect_columns(df, min_max_columns)
log1p_columns = get_intersect_columns(df, log1p_columns)

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import numpy as np

def feature_processing(df):
    # numeric columns
    for column in log1p_columns:
        df[column] = np.log(df[column]+1)
    for column in min_max_columns:
        df[column] = df[column]/df[column].unique().max()
    for column in numeric_columns:
        df[column] = df[column].fillna(df[column].dropna().mean())
    
    scaler = StandardScaler().fit(df[numeric_columns])
    df[numeric_columns] = scaler.transform(df[numeric_columns])
    
    # categorical columns
    labenc = preprocessing.LabelEncoder()
    for column in category_columns:
        df[column] = labenc.fit_transform(df[column].tolist())
    return df

In [ ]:
df = feature_processing(df)

In [ ]:
df[['group1', 'group3', 'group5']] = pd.get_dummies(df['Group'])

In [ ]:
dsp.write_table(df, train_dataset + '_processed', 'feature')

process validate dataset

In [ ]:
df_validate = dsp.read_table(validate_dataset, 'feature')
df_validate = df_validate.loc[df_validate['Group'].isin([1,3,5])]
df_validate = df_validate.loc[df_validate['Range_day']==1]
df_validate = df_validate.drop(missing_columns + col_to_drop, axis=1)
df_validate = feature_processing(df_validate)
df_validate[['group1', 'group3', 'group5']] = pd.get_dummies(df_validate['Group'])
dsp.write_table(df_validate, validate_dataset + '_processed', 'feature')

In [ ]:
df_score = dsp.read_table(score_dataset, 'feature')
df_score = df_score.drop(missing_columns + col_to_drop, axis=1)
df_score = feature_processing(df_score)
dsp.write_table(df_score, score_dataset + '_processed', 'feature')

In [ ]:
import pickle

with open('../pkl/column_list.pickle', 'wb') as f:
    pickle.dump([numeric_columns, category_columns], f)